# This script is to facillitate the upload of fine tuning data for bayes beats

In [2]:
!pip install google-cloud-core google-auth google-auth-oauthlib gcsfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl (181 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 kB 7.2 MB/s eta 0:00:00
  Using cached fsspec-2025.5.0-py3-none-any.whl (196 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Using cached pyasn1-0.6.1-py3-none-any.whl (83 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl (81 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninsta

In [ ]:
from datasets import load_dataset
from google.cloud import storage
import os
from tqdm import tqdm

In [6]:
!gcloud auth application-default login


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=60z9OYq3w3ss5HXp29BxvrendQbH4u&access_type=offline&code_challenge=pFw1San5YFEIjczOk5pLn-q3rKCqhliQ2SIhlVHyVoc&code_challenge_method=S256


Credentials saved to file: [/Users/kwaku/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "pk-arg-prj4-datasci" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this project. You might receive a "quota_exceeded" or "API not enabled" error. Ru

In [ ]:
from datasets import load_dataset
from google.cloud import storage
import soundfile as sf
import io
import json
from tqdm import tqdm

# Config
BATCH_SIZE = 100
MAX_RECORDS = 1000
SPLIT = "train"
BUCKET_NAME = "uchicago-bayesian-bayes-beats"

# GCS
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

# Load dataset
dataset = load_dataset("vikhyatk/lofi", split=SPLIT)

# Batch over the dataset
for start in range(0, min(len(dataset), MAX_RECORDS), BATCH_SIZE):
    end = min(start + BATCH_SIZE, len(dataset))
    batch = dataset.select(range(start, end))

    for i, record in enumerate(tqdm(batch, desc=f"Uploading records {start}–{end}")):
        try:
            audio = record["audio"]
            audio_array = audio["array"]
            sample_rate = audio["sampling_rate"]

            # Trim to 30 seconds
            max_samples = 30 * sample_rate
            trimmed = audio_array[:max_samples]

            # Write WAV to memory
            buffer = io.BytesIO()
            sf.write(buffer, trimmed, sample_rate, format="WAV")
            buffer.seek(0)

            # Generate synthetic file name
            base_name = f"{start + i:06d}"
            blob = bucket.blob(f"vikhyatk/lofi/{base_name}.wav")
            blob.upload_from_file(buffer, content_type="audio/wav")

            # Upload metadata JSON
            metadata = {k: v for k, v in record.items() if k != "audio"}
            meta_blob = bucket.blob(f"vikhyatk/lofi/{base_name}.json")
            meta_blob.upload_from_string(json.dumps(metadata), content_type="application/json")

        except Exception as e:
            print(f"❌ Failed on record {start + i}: {e}")


/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## parallel execution

In [26]:
import os
import zipfile
import requests
import io
import json
from pathlib import Path
from tqdm import tqdm
from pydub import AudioSegment
from google.cloud import storage

# --- CONFIG ---
BUCKET_NAME = "uchicago-bayesian-bayes-beats"
MAX_FILES = 1000
LOCAL_ZIP = "/Users/kwaku/Development/research/bayes_beats/MTG-Jamendo_dataset.zip"
EXTRACT_DIR = Path("jamendo_dataset")
AUDIO_BASE_URL = "https://datasets.i-know-this-song.com/audio/{}.mp3"

# --- SETUP GCS ---
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

# --- VERIFY ZIP ---
import zipfile
if not zipfile.is_zipfile(LOCAL_ZIP):
    raise ValueError("❌ MTG-Jamendo_dataset.zip is not a valid ZIP. Please download using wget as instructed.")

# --- EXTRACT ZIP ---
if not EXTRACT_DIR.exists():
    print("📦 Extracting ZIP...")
    with zipfile.ZipFile(LOCAL_ZIP, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extracted to:", EXTRACT_DIR)

# --- PARSE TRACK IDS FROM split-0.json ---
split_file = EXTRACT_DIR / "MTG-Jamendo_dataset" / "split-0.json"
if not split_file.exists():
    raise FileNotFoundError("❌ split-0.json not found in extracted metadata.")

with open(split_file) as f:
    split_data = json.load(f)

track_ids = [t["id"] for t in split_data["tracks"][:MAX_FILES]]
track_map = {t["id"]: t for t in split_data["tracks"]}

print(f"🎵 Preparing to upload {len(track_ids)} tracks to GCS...")

# --- PROCESS AND UPLOAD AUDIO ---
for track_id in tqdm(track_ids, desc="Uploading tracks"):
    try:
        # Download MP3
        url = AUDIO_BASE_URL.format(track_id)
        response = requests.get(url)
        response.raise_for_status()

        # Load .mp3 into pydub and trim to 30s
        audio = AudioSegment.from_file(io.BytesIO(response.content), format="mp3")
        audio = audio[:30_000]

        # Export to in-memory .wav buffer
        buffer = io.BytesIO()
        audio.export(buffer, format="wav")
        buffer.seek(0)

        # Upload audio
        audio_blob = bucket.blob(f"mtg-jamendo/{track_id}.wav")
        audio_blob.upload_from_file(buffer, content_type="audio/wav")

        # Upload metadata
        metadata = track_map[track_id]
        meta_blob = bucket.blob(f"mtg-jamendo/{track_id}.json")
        meta_blob.upload_from_string(json.dumps(metadata), content_type="application/json")

    except Exception as e:
        print(f"❌ Failed on track {track_id}: {e}")

/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


ValueError: ❌ MTG-Jamendo_dataset.zip is not a valid ZIP. Please download using wget as instructed.

In [27]:
!pip install zenodo-get
!zenodo_get 4984208


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.5/128.5 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: wget is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for wget ... done

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Title: Data from: Kin effects on energy allocation in group-living ground squirrels
Keywords: reproductive allocation, matriline, somatic allocation, Philopatry, 1992-2014, Urocitellus columbianus, energy
Publication date: 2017-04-25
DOI: 10.5061/dryad.50hk0
Total size: 10.3 kB

File: 

In [24]:
!curl -L "https://zenodo.org/record/4984208/files/MTG-Jamendo_dataset.zip?download=1" -o MTG-Jamendo_dataset.zip



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   279  100   279    0     0    274      0  0:00:01  0:00:01 --:--:--   274
100 14305  100 14305    0     0   8694      0  0:00:01  0:00:01 --:--:--     0


In [8]:
!pip install "numpy<2" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 7.6 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.17.2 requires torch==2.2.2, but you have torch 2.1.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
py3langid 0.3.0 requires numpy>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
datasets 3.4.1 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.5.0 which is incompatible.
ace-step 0.2.0 requires diffusers>=0.33.0, but you have diffusers 0.27.2 which is incompatible.
ace-step 0.2.0 requires transformers==4.50.0, but you have transformers 4.37.2 which

In [28]:
import requests
import json
import io
from pathlib import Path
from google.cloud import storage
from tqdm import tqdm

# --- CONFIG ---
BUCKET_NAME = "uchicago-bayesian-bayes-beats"
GCS_FOLDER = "jamendo"
TAG = "rock"
MAX_TRACKS = 20
CLIENT_ID = "a62fe3ee"  # Jamendo test client_id
AUDIO_FORMAT = "mp32"   # 320kbps

# --- SETUP GCS ---
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

# --- FETCH TRACKS ---
print(f"🎵 Fetching up to {MAX_TRACKS} '{TAG}' tracks from Jamendo...")
response = requests.get(
    "https://api.jamendo.com/v3.0/tracks",
    params={
        "client_id": CLIENT_ID,
        "format": "json",
        "limit": MAX_TRACKS,
        "audioformat": AUDIO_FORMAT,
        "include": "musicinfo",
        "tags": TAG
    }
)
tracks = response.json()["results"]
print(f"✅ Retrieved {len(tracks)} tracks")

# --- PROCESS TRACKS ---
for i, track in enumerate(tqdm(tracks, desc="Uploading to GCS")):
    try:
        title = track["name"].replace(" ", "_")
        artist = track["artist_name"].replace(" ", "_")
        filename = f"{i:03d}_{artist}_{title}.mp3"

        # Download MP3 to memory
        audio_resp = requests.get(track["audio"])
        audio_resp.raise_for_status()
        buffer = io.BytesIO(audio_resp.content)

        # Upload MP3 to GCS
        audio_blob = bucket.blob(f"{GCS_FOLDER}/{filename}")
        audio_blob.upload_from_file(buffer, content_type="audio/mpeg")

        # Upload metadata as JSON
        metadata = {
            "id": track["id"],
            "name": track["name"],
            "artist": track["artist_name"],
            "album": track.get("album_name"),
            "duration": track["duration"],
            "tags": track.get("musicinfo", {}).get("tags", []),
            "genres": track.get("musicinfo", {}).get("genres", []),
            "instruments": track.get("musicinfo", {}).get("instruments", []),
            "vocalinstrumental": track.get("musicinfo", {}).get("vocalinstrumental", None),
        }
        meta_blob = bucket.blob(f"{GCS_FOLDER}/{filename.replace('.mp3', '.json')}")
        meta_blob.upload_from_string(json.dumps(metadata), content_type="application/json")

    except Exception as e:
        print(f"❌ Failed on {track['name']}: {e}")


/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


🎵 Fetching up to 20 'rock' tracks from Jamendo...
✅ Retrieved 20 tracks


Uploading to GCS: 100%|██████████| 20/20 [03:46<00:00, 11.33s/it]


In [1]:
import requests, json, io
from google.cloud import storage
from tqdm import tqdm

# --- Config ---
BUCKET_NAME = "uchicago-bayesian-bayes-beats"
BASE_FOLDER = "jamendo_by_genre"
CLIENT_ID = "a62fe3ee"
AUDIO_FORMAT = "mp32"
TRACKS_PER_GENRE = 20



# --- Initialize GCS ---
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

# "rock", "pop",
GENRES = [
     "electronic", "jazz", "hiphop",
    "classical", "metal", "soundtrack", "relaxation", "world",
    "blues", "funk", "reggae", "latin", "ambient",
    "folk", "techno", "punk", "house", "experimental"
]

# --- Loop by Genre ---
for genre in GENRES:
    print(f"\n🎧 Processing genre: {genre}")
    params = {
        "client_id": CLIENT_ID,
        "format": "json",
        "limit": TRACKS_PER_GENRE,
        "audioformat": AUDIO_FORMAT,
        "include": "musicinfo",
        "tags": genre
    }
    resp = requests.get("https://api.jamendo.com/v3.0/tracks", params=params)
    tracks = resp.json().get("results", [])
    print(f"→ Retrieved {len(tracks)} tracks for {genre}")

    for i, track in enumerate(tracks):
        try:
            title = track["name"].replace(" ", "_")
            artist = track["artist_name"].replace(" ", "_")
            filename = f"{i:02d}_{artist}_{title}.mp3"

            # Download MP3
            audio_resp = requests.get(track["audio"])
            audio_resp.raise_for_status()
            buffer = io.BytesIO(audio_resp.content)
            buffer.seek(0)

            # Upload MP3
            audio_blob = bucket.blob(f"{BASE_FOLDER}/{genre}/{filename}")
            audio_blob.upload_from_file(buffer, content_type="audio/mpeg")

            # Upload metadata JSON
            metadata = {
                "id": track["id"],
                "name": track["name"],
                "artist": track["artist_name"],
                "album": track.get("album_name"),
                "duration": track["duration"],
                "tags": track.get("musicinfo", {}).get("tags", []),
                "genres": track.get("musicinfo", {}).get("genres", []),
                "instruments": track.get("musicinfo", {}).get("instruments", []),
                "vocalinstrumental": track.get("musicinfo", {}).get("vocalinstrumental")
            }
            meta_blob = bucket.blob(f"{BASE_FOLDER}/{genre}/{filename.replace('.mp3','.json')}")
            meta_blob.upload_from_string(json.dumps(metadata), content_type="application/json")

        except Exception as e:
            print(f"❌ Error uploading track {i} for genre {genre}: {e}")


/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kwaku/.pyenv/versions/3.10.13/envs/bayes-final/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)



🎧 Processing genre: electronic
→ Retrieved 20 tracks for electronic

🎧 Processing genre: jazz
→ Retrieved 20 tracks for jazz

🎧 Processing genre: hiphop
→ Retrieved 20 tracks for hiphop

🎧 Processing genre: classical
→ Retrieved 20 tracks for classical

🎧 Processing genre: metal
→ Retrieved 20 tracks for metal

🎧 Processing genre: soundtrack
→ Retrieved 20 tracks for soundtrack

🎧 Processing genre: relaxation
→ Retrieved 20 tracks for relaxation

🎧 Processing genre: world
→ Retrieved 20 tracks for world

🎧 Processing genre: blues
→ Retrieved 20 tracks for blues

🎧 Processing genre: funk
→ Retrieved 20 tracks for funk

🎧 Processing genre: reggae
→ Retrieved 20 tracks for reggae

🎧 Processing genre: latin
→ Retrieved 20 tracks for latin

🎧 Processing genre: ambient
→ Retrieved 20 tracks for ambient
❌ Error uploading track 10 for genre ambient: Timeout of 120.0s exceeded, last exception: HTTPSConnectionPool(host='storage.googleapis.com', port=443): Max retries exceeded with url: /upload/

ConnectionError: HTTPSConnectionPool(host='api.jamendo.com', port=443): Max retries exceeded with url: /v3.0/tracks?client_id=a62fe3ee&format=json&limit=20&audioformat=mp32&include=musicinfo&tags=folk (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10680f040>: Failed to resolve 'api.jamendo.com' ([Errno 8] nodename nor servname provided, or not known)"))